In [6]:
import warnings
import os
warnings.filterwarnings('ignore')

In [7]:
#Reading the API Key

with open('../openai_api_key.txt', 'r') as f:
    api_key = f.read()
os.environ['OPENAI_API_KEY'] = api_key

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

### Response Caching

In [9]:
from langchain.globals import set_llm_cache
chat = ChatOpenAI()


In [10]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

# First request, it is not yet in cache, so it should take longer
messages = [HumanMessage(content = 'Tell me a joke')]
response = chat(messages = messages, max_tokens = 50)
print(response.content)

Why couldn't the bicycle find its way home? Because it lost its bearings!
CPU times: total: 812 ms
Wall time: 2.76 s


In [11]:
%%time
# Second request is faster, as response is cached
messages = [HumanMessage(content = 'Tell me a joke')]
response = chat(messages = messages, max_tokens = 50)
print(response.content)

Why couldn't the bicycle find its way home? Because it lost its bearings!
CPU times: total: 15.6 ms
Wall time: 2 ms


### Prompt Templating

In [12]:
# Using Format strings
from langchain.llms import OpenAI
llm = OpenAI()

prompt_template = "Tell me something about {topic}"
print(llm(prompt_template.format(topic='Insurance')))

# Insurance is a contract between an individual or organization (the insured) and an insurance company (the insurer) where the insurer agrees to provide financial compensation for specified losses, damages, or injuries in exchange for a premium. The purpose of insurance is to protect against the risk of financial loss due to unexpected events such as accidents, illness, natural disasters, or death. Insurance can cover various aspects of life, including health, property, life, and liability. The amount of coverage and the cost of premiums depend on factors such as the type of insurance, the level of risk, and the individual's age and health. Insurance is typically required for certain activities, such as driving a car or owning a home, and can provide peace of mind and financial security for individuals and businesses.



Insurance is a contract between an individual or entity (the policyholder) and an insurance company, where the policyholder pays a premium in exchange for protection against potential financial losses. The insurance company agrees to cover the costs of certain risks, such as property damage, illness, or death, as outlined in the insurance policy.

There are many types of insurance, including life, health, property, auto, and liability insurance. Each type of insurance covers different risks and provides varying levels of coverage. For example, life insurance provides financial support for the policyholder's loved ones in the event of their death, while health insurance covers medical expenses.

Insurance works on the principle of risk pooling, where a large group of people pay premiums into a pool, and the insurance company uses those funds to pay out claims to the few who experience a covered loss. This allows individuals to protect themselves against potential financial losses with

In [13]:
# Using f-string literals

topic = 'Insurance'
prompt_template = f"Tell me something about {topic}"
print(llm(prompt_template))

# Insurance is a contract between an individual or organization (the insured) and an insurance company (the insurer) where the insurer agrees to provide financial compensation for specified losses, damages, or injuries in exchange for a premium. The purpose of insurance is to protect against the risk of financial loss due to unexpected events such as accidents, illness, natural disasters, or death. Insurance can cover various aspects of life, including health, property, life, and liability. The amount of coverage and the cost of premiums depend on factors such as the type of insurance, the level of risk, and the individual's age and health. Insurance is typically required for certain activities, such as driving a car or owning a home, and can provide peace of mind and financial security for individuals and businesses.



Insurance is a contract between an individual or entity (the policyholder) and an insurance company, where the policyholder pays a premium in exchange for protection against potential financial losses. The insurance company agrees to cover the costs of certain risks, such as property damage, illness, or death, as outlined in the insurance policy.

There are many types of insurance, including life, health, property, auto, and liability insurance. Each type of insurance covers different risks and provides varying levels of coverage. For example, life insurance provides financial support for the policyholder's loved ones in the event of their death, while health insurance covers medical expenses.

Insurance works on the principle of risk pooling, where a large group of people pay premiums into a pool, and the insurance company uses those funds to pay out claims to the few who experience a covered loss. This allows individuals to protect themselves against potential financial losses with

In [14]:
def get_prompt(topic):
    prompt = f" Tell me something about {topic}"
    return prompt
get_prompt(topic = 'data science')

' Tell me something about data science'

Prompt Templating using langchain Prompt Template

In [15]:
# Using format strings with langchain schema
from langchain.schema import SystemMessage
prompt_template = "Tell me something about {topic}"
system_prompt = SystemMessage(prompt_template.format(topic = 'Insurance'))
print(llm(system_prompt.content))




Insurance is a contract between an individual or entity (the policyholder) and an insurance company, where the policyholder pays a premium in exchange for protection against potential financial losses. The insurance company agrees to cover the costs of certain risks, such as property damage, illness, or death, as outlined in the insurance policy.

There are many types of insurance, including life, health, property, auto, and liability insurance. Each type of insurance covers different risks and provides varying levels of coverage. For example, life insurance provides financial support for the policyholder's loved ones in the event of their death, while health insurance covers medical expenses.

Insurance works on the principle of risk pooling, where a large group of people pay premiums into a pool, and the insurance company uses those funds to pay out claims to the few who experience a covered loss. This allows individuals to protect themselves against potential financial losses with

In [16]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(
    input_variables = [''],
    template = "Tell me something about {topic}"
)
prompt = prompt_template.format(topic = 'Data Science')
prompt

'Tell me something about Data Science'

In [17]:
# adding more elements 
prompt_template = PromptTemplate(
    input_variables=['topic','num_words'],
    template="Tell me something about {topic} in {num_words} words"
)
prompt = prompt_template.format(topic = 'Data Science',num_words = 30)
prompt

'Tell me something about Data Science in 30 words'

Prompt Templating using langchain prompt templates

In [18]:
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate

human_template = "Tell me something about {topic}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

prompt = chat_prompt.format_prompt(topic = 'Insurance')
print(prompt)

# to get the messages from ChatPromptValue
messages = prompt.messages
print(messages)

# getting response from chat model
response = chat(messages = messages)
response



messages=[HumanMessage(content='Tell me something about Insurance')]
[HumanMessage(content='Tell me something about Insurance')]


AIMessage(content='Insurance is a contract between an individual or organization (the policyholder) and an insurance company, in which the insurance company agrees to provide financial protection or reimbursement for specified losses or damages in exchange for the payment of a premium. There are many different types of insurance, including health insurance, life insurance, auto insurance, and property insurance. Insurance helps individuals and businesses manage risk and protect themselves from financial losses in the event of an unexpected event or disaster.')

### 2nd example

In [19]:
from langchain.prompts.chat import HumanMessagePromptTemplate,SystemMessagePromptTemplate,ChatPromptTemplate

##### System Message Prompt Template

In [20]:
system_template = "You are a {character}"
system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
system_message_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character'], template='You are a {character}'))

#### Human Message Prompt Template

In [21]:
human_template = "Write a crime scene involving {item1} in a {item2}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
human_message_prompt

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['item1', 'item2'], template='Write a crime scene involving {item1} in a {item2}'))

#### Chat Prompt Template

In [22]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['character', 'item1', 'item2'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character'], template='You are a {character}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['item1', 'item2'], template='Write a crime scene involving {item1} in a {item2}'))])

In [23]:
prompt = chat_prompt.format_prompt(character = 'Writer', item1 = 'robbery', item2 = 'mansion')
prompt.messages

[SystemMessage(content='You are a Writer'),
 HumanMessage(content='Write a crime scene involving robbery in a mansion')]

In [24]:
response = chat(messages = prompt.messages, max_tokens = 100)
response.content

'The grand mansion stood silent and imposing in the moonlit night, its ornate gates creaking softly in the wind. Inside, a lavish party had been in full swing, with guests sipping champagne and dancing to the music echoing through the opulent halls. But now, the mansion was shrouded in darkness, the only sound the faint rustling of leaves outside.\n\nDetective Jameson stepped out of his car, the flashing lights of the police cars illuminating the scene in an eerie'

### Prompt Serialization

In [25]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(
    input_variables = [''],
    template = "Tell me something about {topic}"
)
prompt_template.save('prompt.json')

In [26]:
import json
f = open('prompt.json')
data = json.load(f)
data

{'name': None,
 'input_variables': ['topic'],
 'input_types': {},
 'output_parser': None,
 'partial_variables': {},
 'metadata': None,
 'tags': None,
 'template': 'Tell me something about {topic}',
 'template_format': 'f-string',
 'validate_template': False,
 '_type': 'prompt'}

In [27]:
# loading the prompt template
from langchain.prompts import load_prompt
loaded_prompt = load_prompt('prompt.json')
loaded_prompt

PromptTemplate(input_variables=['topic'], template='Tell me something about {topic}')